In [2]:
'''An analysis of how to compute the logarithm of the quantity ξ in Calvetti et. al. , uses the complex valued logarithm. 
As we end up exponentiating the result, Euler's identity results in a number of sign changes, but the results are identical to the lin domain. '''




import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import numpy as np
from utilities.Utils import jacob



rng = np.random.default_rng(5)
theta = np.array([rng.uniform(0.,1.) for _ in range(10)])
weights = np.array([rng.uniform(0.,1.) for _ in range(10)])
weights = weights / np.sum(weights)


'''Computing ln(sum(weights * ln(theta)))'''



expectation_log_theta = 0
for i in range(len(theta)): 
    expectation_log_theta += weights[i] * np.log(theta[i])

print(f"Using the naive approach: {expectation_log_theta}")

deltas = np.zeros(len(theta))

for i in range(len(deltas)): 
    log_theta = np.log(theta[i])
    log_log = np.log(np.abs(log_theta))
    deltas[i] = np.log(weights[i]) + log_log


print(f"ξ using the complex valued log: {-1 * np.exp(jacob(deltas)[-1])}")

ξ = -1 * np.exp(jacob(deltas)[-1])








Using the naive approach: -1.5990854435992887
ξ using the complex valued log: -1.599085443599289


In [3]:
'''How to compute the logarithm of the quantity Σ in Calvetti et al. using the complex valued logarithm and matrix exponentials.
We have the quantity ξ from above and will resuse it here. '''


psis = np.array(np.log(theta) - ξ)

deltas = np.zeros(len(theta))
for i in range(len(deltas)): 
    deltas[i] = np.log(weights[i]) + np.log(psis[i] ** 2)

print(deltas)

cov = 0
for i in range(len(theta)):
    cov += weights[i] * ((np.log(theta[i]) - ξ) ** 2)
    

print(f"Σ using the naive approach: {cov}")
print(f"Σ using the complex valued log: {np.exp(jacob(deltas)[-1])}")

'''This only covers theta as a single value, we need a slightly more advanced approach for higher dimensional theta vectors'''















[-1.51339906 -2.53124622 -2.69810631 -3.87873589 -1.28478435 -2.79445164
 -3.37167998 -1.63530179 -1.42025905 -3.59586082]
Σ using the naive approach: 1.2239025881496943
Σ using the complex valued log: 1.2239025881496943


'This only covers theta as a single value, we need a slightly more advanced approach for higher dimensional theta vectors'

In [12]:
'''For high dimensional theta, some log terms in the covariance matrix may be negative, requiring us to handle each case differently'''

N = 10
rng = np.random.default_rng(5)
theta = np.array([[rng.uniform(0.,1.) for _ in range(3)] for _ in range(N)])
weights = np.array([rng.uniform(0.,1.) for _ in range(N)])
weights = weights / np.sum(weights)

expectation_log_theta = 0
for i in range(len(theta)): 
    expectation_log_theta += weights[i] * np.log(theta[i])

print(f"Using the naive approach: {expectation_log_theta}")



deltas = np.zeros((N,3))


'''Theta matrix needs to be shape num_particles * len(theta)'''
for i in range(theta.shape[1]): 
    sub_theta = theta[:,i]
    for j in range(theta.shape[0]):
        log_theta = np.log(sub_theta[j])
        log_log = np.log(np.abs(log_theta))

        deltas[j,i] = np.log(weights[j]) + log_log


ξ = []
for i in range(theta.shape[1]): 
   ξ.append(-1 * np.exp(jacob(deltas[:,i])))

ξ = np.array(ξ)[:,-1]

print(f"ξ using the complex valued log: {ξ}")

'''Now for Σ '''
psis = np.array(np.log(theta) - ξ)


Σ = 0
for i in range(N):
    Σ += weights[i] * np.outer(psis[i,:] - ξ,psis[i,:]-ξ)

print(f"ξ using the naive approach: {Σ}")





Using the naive approach: [-0.40812203 -1.11460911 -1.74232264]
ξ using the complex valued log: [-0.40812203 -1.11460911 -1.74232264]
ξ using the lin domain: [[0.30882339 0.58863307 0.77967427]
 [0.58863307 2.79004821 1.7230248 ]
 [0.77967427 1.7230248  4.84550668]]
